In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
from utils import imputer, shuffle_data_by_group, reset_group_id, datetime_to_days_diff, train_test_split_grouped_extrapolation, train_test_split_grouped_interpolation

In [2]:
df1 = pd.read_csv('patras_data.csv', parse_dates=['date'])
df2 = pd.read_csv('sheffield_data.csv', parse_dates=['date'])

# Combine dataframes

In [3]:
# Reset Patient ID for both dataframes
df1.groupby(['ID']).ngroup()

0         0
1         0
2         0
3         0
4         0
       ... 
4348    558
4349    558
4350    558
4351    559
4352    559
Length: 4353, dtype: int64

In [4]:
df1['ID'] = df1.groupby(['ID']).ngroup()

In [5]:
df2.groupby(['ID']).ngroup() + df1['ID'].unique().size

0        560
1        560
2        560
3        560
4        560
        ... 
6637    1552
6638    1552
6639    1553
6640    1553
6641    1553
Length: 6642, dtype: int64

In [6]:
df2['ID'] = df2.groupby(['ID']).ngroup() + df1['ID'].unique().size

In [7]:
data = pd.concat([df1, df2], axis=0, ignore_index=True)

In [8]:
data

,ID,site,date_entered_study,control_patient,dob_year,gender,ethnicity,employment,height,smoker,...,neuropathy,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe,AKI,Started_dialysis
0,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10990,1552,Sheffield,09/06/2019,False,1973.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10991,1552,Sheffield,09/06/2019,False,1973.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
10992,1553,Sheffield,09/06/2019,False,1948.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10993,1553,Sheffield,09/06/2019,False,1948.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [9]:
data.describe()

,ID,dob_year,height,sampleId,egfr,ckd_stage,slope_allprior,N_allprior,slope_2yprior,N_2yprior,...,micro_albuminuria,hba1c,c_reactive_protein,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe
count,10995.000000,8924.000000,10985.000000,3851.000000,10522.000000,3847.000000,9024.000000,10316.000000,8916.000000,10316.000000,...,187.000000,286.000000,234.000000,603.000000,599.000000,599.000000,155.000000,154.000000,340.000000,292.000000
mean,704.031924,1959.437584,5.387872,1016.242534,46.771944,1.408630,0.034643,5.451241,-0.054014,4.063494,...,1555.666695,23.944965,2.914231,57.589552,2628.758097,1514.981636,410.993548,9.491558,48.803235,207.782192
std,459.267671,15.180295,27.448790,601.277980,25.192801,1.722691,67.007580,3.621561,68.600263,2.270368,...,2309.599102,26.783405,9.077228,55.430334,3926.904331,18566.007661,235.477892,22.940807,37.259726,125.553761
min,0.000000,1926.000000,0.000000,1.000000,0.000000,0.000000,-1460.000000,1.000000,-1460.000000,1.000000,...,0.000000,4.600000,0.000000,7.900000,3.700000,22.000000,50.000000,1.700000,4.300000,3.800000
25%,276.000000,1948.000000,0.000000,510.500000,27.000000,0.000000,-5.318589,2.000000,-5.903179,2.000000,...,58.750000,6.400000,0.230000,11.900000,7.290000,187.500000,265.500000,4.025000,14.950000,54.200000
50%,711.000000,1957.000000,1.560000,997.000000,42.000000,0.000000,-1.031236,5.000000,-1.170697,4.000000,...,545.000000,7.750000,0.435000,14.600000,9.820000,238.000000,379.000000,5.550000,46.000000,254.500000
75%,1098.000000,1969.000000,1.700000,1481.500000,61.000000,3.000000,2.282602,8.000000,3.055405,5.000000,...,2326.500000,47.000000,1.000000,116.000000,6305.000000,284.000000,494.500000,8.725000,71.000000,306.000000
max,1553.000000,2002.000000,188.000000,2274.000000,189.000000,41.000000,4015.000000,17.000000,4015.000000,14.000000,...,15171.000000,137.000000,81.000000,166.000000,16270.000000,320000.000000,2000.000000,284.000000,342.000000,510.000000


In [10]:
# Select non-control patients
data = data[data['control_patient']==False]
data.reset_index(inplace=True, drop=True)
data

,ID,site,date_entered_study,control_patient,dob_year,gender,ethnicity,employment,height,smoker,...,neuropathy,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe,AKI,Started_dialysis
0,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Patras,16/02/2016 17:37,False,1946.0,Male,Caucasian,Unemployed,1.7,Non-Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10814,1552,Sheffield,09/06/2019,False,1973.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10815,1552,Sheffield,09/06/2019,False,1973.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
10816,1553,Sheffield,09/06/2019,False,1948.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10817,1553,Sheffield,09/06/2019,False,1948.0,Female,Caucasian,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


# Cleaning not involving imputing values

## Initial cleaning

In [11]:
# Dataframe after selecting relevant columns
data2 = data[['ID', 'site', 'dob_year','gender', 'ethnicity', 'height', 'Weight', 'smoker', 'kidney_transplant', 'Patient_died', 'disease', 'bp.sys', 'bun', 'date', 'egfr']]
data2 = data2.rename(columns={'Weight': 'weight', 'Patient_died': 'patient_died'})
data2['ID'] = data2.groupby(['ID']).ngroup()
data2

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10814,1421,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0
10815,1421,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0
10816,1422,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0
10817,1422,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0


In [12]:
data2['smoker'] = data2['smoker'].replace({'Past Smoker': 'Past-Smoker'})

In [13]:
data2['smoker'].value_counts()

Non-Smoker     2602
Past-Smoker    1567
Smoker          896
Name: smoker, dtype: int64

In [14]:
# Notice that for this patient, egfr and ckd_stage appears to have been switched incorrectly, so we manualy set egfr to the correct value
data.loc[data['egfr']==3]

,ID,site,date_entered_study,control_patient,dob_year,gender,ethnicity,employment,height,smoker,...,neuropathy,haemaglobin,wbc,platelets,vitamin_b12,folate,serum_fe,total_fe,AKI,Started_dialysis
3539,373,Patras,17/07/2017 12:39,False,1956.0,Male,Caucasian,Semi-Professional,1.7,Smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [15]:
data2.loc[data2['egfr']==3, 'egfr'] = 41

In [16]:
# Sort values by date
data3 = data2.sort_values(['ID', 'date'], ignore_index=True)

## Regrouping ethnicity

In [17]:
data4 = data3.copy()

In [18]:
data4['ethnicity'].value_counts()

Caucasian                        9209
Asian                             146
Black (afro caribean descent)     108
Others                             56
Asian - Indian                     12
INDIAN                             11
Asian (Pakistan)                   11
Somali                              9
Asian(INDIAN)                       6
Chinese                             6
Asian( PAKISTANI)                   5
Indian                              5
Asian ( BANGLADESHHI)               2
Arab                                1
Name: ethnicity, dtype: int64

In [19]:
# Combine categories
data4['ethnicity'] = data4['ethnicity'].replace(dict.fromkeys(['Asian - Indian', 'Asian (Pakistan)', 'INDIAN', 'Chinese', 'Asian(INDIAN)', 'Indian', 'Asian( PAKISTANI)', 'Arab', 'Asian ( BANGLADESHHI)'], 'Asian'))
data4['ethnicity'] = data4['ethnicity'].replace({'Somali': 'Others'})
data4['ethnicity'] = data4['ethnicity'].replace({'Black (afro caribean descent)': 'Black'})

In [20]:
data4['ethnicity'].value_counts()

Caucasian    9209
Asian         205
Black         108
Others         65
Name: ethnicity, dtype: int64

## egfr

In [21]:
data4['egfr'].unique()

array([ 69. ,  62. ,  56. ,  50. ,  54. ,  53. ,  46. ,  58. ,  49. ,
        90. , 103. ,  33. ,  31. ,  29. ,  36. ,  35. ,  45. ,  48. ,
        59. ,  72. ,  41. ,  44. ,  64. ,  60. ,  71. ,  47. ,  66. ,
        76. ,  52. ,  39. ,  28. ,  84. ,  89. ,  24. ,  38. ,  27. ,
        21. ,  22. ,  30. ,  23. ,  20. ,  19. ,  26. ,  16. ,  13. ,
        32. ,  34. ,  43. ,  37. ,  25. ,   9. ,  15. ,  10. , 107. ,
        82. ,  93. ,  63. , 106. ,  92. , 118. , 150. , 149. , 101. ,
       125. , 127. , 124. ,  67. ,  55. ,  40. ,  57. ,  68. ,  42. ,
        65. ,  73. ,  80. ,  81. ,  61. ,  18. ,  78. ,  70. ,  14. ,
        17. ,  11. ,  12. ,  85. ,  77. ,  79. , 102. ,  74. ,  51. ,
        97. ,  83. ,  96. , 108. ,  94. ,  86. ,  75. , 109. ,  91. ,
       104. ,  88. ,  87. ,  95. , 114. , 100. , 133. ,  99. , 113. ,
       112. ,   7. , 120. ,  98. , 158. , 128. , 129. , 145. , 122. ,
       111. , 117. , 110. , 131. ,   8. , 123. , 146. , 105. , 136. ,
       119. , 141. ,

In [22]:
# Missing values for egfr
data4['egfr'].isna().sum()

402

In [23]:
# Drop all rows with missing egfr values
data5 = data4.dropna(subset=['egfr'])
data5.reset_index(inplace=True, drop=True)
data5['ID'] = data5.groupby(['ID']).ngroup()

C:\Users\nick_\AppData\Local\Temp/ipykernel_1720/3936683330.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data5['ID'] = data5.groupby(['ID']).ngroup()


In [24]:
# Drop row with 0 egfr value (1 row)
data6 = data5.loc[data5['egfr'] != 0]
data6.reset_index(inplace=True, drop=True)
data6['ID'] = data6.groupby(['ID']).ngroup()
data6

C:\Users\nick_\AppData\Local\Temp/ipykernel_1720/1500698124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data6['ID'] = data6.groupby(['ID']).ngroup()


,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10411,1370,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0
10412,1370,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0
10413,1371,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0
10414,1371,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0


In [25]:
data6

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10411,1370,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0
10412,1370,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0
10413,1371,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0
10414,1371,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0


In [26]:
data7 = datetime_to_days_diff(data6, 'ID', 'date')

c:\Users\nick_\OneDrive\Desktop\Thesis code\my_work\clustering\utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['times'] = None
c:\Users\nick_\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\nick_\OneDrive\Desktop\Thesis code\my_work\clustering\utils.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [39]:
data7

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr,times
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0,0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0,449
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0,579
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0,716
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0,891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10411,1370,Sheffield,1973.0,Female,Caucasian,0.0,NaN,NaN,False,False,Other,137.0,5.0,2019-06-13,65.0,0
10412,1370,Sheffield,1973.0,Female,Caucasian,0.0,76.6,NaN,False,False,Other,139.0,4.5,2019-12-09,64.0,179
10413,1371,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0,0
10414,1371,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0,137


In [55]:
# Select patients with at least 3 datapoints
min_vals = 3
data8 = data7[data7['ID'].isin(np.where(data7['ID'].value_counts(sort=False).values >= min_vals)[0])]
data8

,ID,site,dob_year,gender,ethnicity,height,weight,smoker,kidney_transplant,patient_died,disease,bp.sys,bun,date,egfr,times
0,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,150.0,47.0,2011-04-10,69.0,0
1,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,0.0,64.0,2012-07-02,62.0,449
2,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,145.0,64.0,2012-11-09,56.0,579
3,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,160.0,55.0,2013-03-26,62.0,716
4,0,Patras,1946.0,Male,Caucasian,1.7,NaN,Non-Smoker,False,False,DN,165.0,54.0,2013-09-17,50.0,891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10409,1369,Sheffield,1954.0,Male,Caucasian,0.0,NaN,NaN,False,False,DN,190.0,11.2,2019-06-09,36.0,689
10410,1369,Sheffield,1954.0,Male,Caucasian,0.0,140.8,NaN,False,False,DN,163.0,9.6,2019-10-18,40.0,820
10413,1371,Sheffield,1948.0,Female,Caucasian,0.0,NaN,NaN,False,False,HTN,0.0,29.0,2019-07-25,5.0,0
10414,1371,Sheffield,1948.0,Female,Caucasian,0.0,90.0,NaN,False,False,HTN,165.0,12.3,2019-12-09,32.0,137


In [58]:
data9 = data8[['ID', 'egfr', 'times']]
data9

,ID,egfr,times
0,0,69.0,0
1,0,62.0,449
2,0,56.0,579
3,0,62.0,716
4,0,50.0,891
...,...,...,...
10409,1369,36.0,689
10410,1369,40.0,820
10413,1371,5.0,0
10414,1371,32.0,137


In [59]:
data9.to_csv('clustering_data.csv')